# 올리브영 

- 샴푸 데이터 560개 수집
- 리뷰 개수는 1000만 표시 

## goodsNo 수집 

In [ ]:
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import urllib.parse as urlparse
from bs4 import BeautifulSoup
import time
import random

def human_like_sleep(min_sec=1.0, max_sec=2.0):
    time.sleep(random.uniform(min_sec, max_sec))

driver = uc.Chrome()

base_url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=1000001000400080001&fltDispCatNo=&prdSort=01&pageIdx={}&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat1000001000400080001_Small"

goodsNos = []

for page in range(1, 25):
    url = base_url.format(page)
    print(f"{page}페이지 이동 중...")
    driver.get(url)
    human_like_sleep()

    soup = BeautifulSoup(driver.page_source, "html.parser")
    links = soup.select(".prd_info .prd_name a")

    for link in links:
        href = link.get("href")
        if href:
            parsed = urlparse.urlparse(href)
            params = urlparse.parse_qs(parsed.query)
            if 'goodsNo' in params:
                goodsNo = params['goodsNo'][0]
                if goodsNo not in goodsNos:
                    goodsNos.append(goodsNo)
                    print(f"goodsNo 수집: {goodsNo}")

print(f"\n총 수집된 goodsNo: {len(goodsNos)}개")
driver.quit()

## 전체 데이터 수집 

- ./shampoo/ 폴더 만들어야 함 

In [ ]:
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

for goodsno in range(2):
    print(f"\n상품 {goodsno} 수집 시작")

    driver = uc.Chrome()
    url = f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={goodsNos[goodsno]}"
    driver.get(url)
    driver.implicitly_wait(5)
    time.sleep(random.uniform(1.5, 2.5)) 

    try:
        driver.find_element(By.CSS_SELECTOR, "#reviewInfo a").click()
        time.sleep(random.uniform(2.0, 3.0))
    except:
        print(f"상품 {goodsno} 리뷰 탭 클릭 실패")
        driver.quit()
        continue

    review_data = []

    def extract_reviews(soup):
        meta = {
            "제품명": soup.select_one("#Contents .prd_name").text.strip(),
            "브랜드": soup.select_one("#Contents .prd_brand a").text.strip(),
            "가격1": soup.select_one("#Contents .price-1").text.strip() if soup.select_one("#Contents .price-1") else None,
            "가격2": soup.select_one("#Contents .price-2").text.strip() if soup.select_one("#Contents .price-2") else None,
            "리뷰수": soup.select_one(".product_rating_area .total em").text.strip() if soup.select_one(".product_rating_area .total em") else None,
            "평균 별점": soup.select_one(".product_rating_area .num strong").text.strip() if soup.select_one(".product_rating_area .num strong") else None,
        }

        리뷰리스트 = [i.text.strip() for i in soup.select("#gdasList .txt_inner")]
        별점리스트 = [i.text.strip() for i in soup.select("#gdasList .review_point span")]

        for 리뷰, 별점 in zip(리뷰리스트, 별점리스트):
            review_data.append({
                **meta,
                "리뷰": 리뷰,
                "별점": 별점
            })

    soup = BeautifulSoup(driver.page_source, "html.parser")
    extract_reviews(soup)
    print("1페이지 수집 완료")

    current_page = 2
    while True:
        try:
            time.sleep(random.uniform(1.0, 2.0))
            page_links = driver.find_elements(By.CSS_SELECTOR, ".pageing a[data-page-no]")
    
            clicked = False
            for link in page_links:
                page_no = int(link.get_attribute("data-page-no"))
                if page_no == current_page:
                    driver.execute_script("arguments[0].click();", link)
                    time.sleep(random.uniform(1.5, 2.5))
                    soup = BeautifulSoup(driver.page_source, "html.parser")
                    extract_reviews(soup)
                    print(f"{current_page}페이지 수집 완료")
                    current_page += 1
                    clicked = True
                    break
    
            if not clicked:
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, ".pageing a.next")
                    driver.execute_script("arguments[0].click();", next_button)
                    time.sleep(random.uniform(1.5, 2.5))
                except:
                    print("더 이상 다음 페이지 없음")
                    break
    
        except Exception as e:
            print(f"오류 발생 - {current_page}페이지: {e}")
            break

    driver.quit()

    if review_data:
        df = pd.DataFrame(review_data)
        df.to_csv(f"./shampoo/oliveyoung_product_{goodsno}.csv", index=False, encoding="utf-8-sig")
        print(f"상품 {goodsno} 리뷰 {len(df)}개 저장 완료")
    else:
        print(f"상품 {goodsno} 리뷰 없음")